In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

In [3]:
# #Replacing Null values of Company_type to Other
data['company_type'].fillna('NpO', inplace = True)
test['company_type'].fillna('NpO', inplace = True)

#replacing with medium company size
data['company_size'].fillna('50-99', inplace  = True)
test['company_size'].fillna('50-99', inplace  = True)

# # #Managing all values accordingly
data.experience[data.experience == '>20'] = 20 
data.experience[data.experience == '<1'] = 1 

test.experience[test.experience == '>20'] = 20 
test.experience[test.experience == '<1'] = 1 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(2, inplace = True)
test['last_new_job'].fillna(2, inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna('Masters', inplace = True)
test['education_level'].fillna('Masters', inplace = True) 

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna('Part time course', inplace = True)
test['enrolled_university'].fillna('Part time course', inplace = True)

# #Filling NULL values with mode
data.gender.fillna('Other', inplace =  True) #data['gender'].mode()[0]
test.gender.fillna('Other', inplace =  True) #test['gender'].mode()[0]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [4]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,Other,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,Other,No relevent experience,no_enrollment,Masters,STEM,8,50-99,NpO,2,88,1


In [5]:
X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

In [6]:
from sklearn import preprocessing

for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))

In [7]:
X.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,23798,41,0.689,1,0,2,0,5,13,1,6,0,106
1,29166,112,0.923,1,0,2,0,5,5,7,1,0,69
2,46,48,0.910,2,0,2,0,5,16,4,5,1,4
3,18527,93,0.666,1,0,2,0,5,5,4,6,0,26
4,21751,2,0.887,2,1,2,2,5,18,4,3,1,88


One Hot Encoding

In [8]:


X = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X['education_level'], prefix='education_level'),
                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X['company_size'], prefix='company_size'),
                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)

X_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X_test['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X_test['education_level'], prefix='education_level'),
                           pd.get_dummies(X_test['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X_test['company_size'], prefix='company_size'),
                           pd.get_dummies(X_test['company_type'], prefix='company_type')],axis=1)

# XGBM

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

In [10]:
params = {
    'min_child_weight': 20.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'max_delta_step': 5.0,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.0,
    'num_boost_round' : 2000
    }

In [11]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

In [12]:
from sklearn.model_selection import StratifiedKFold
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

In [13]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print('[Fold %d/%d]' % (i + 1, kfold))
   
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_val, y_val)
    d_test = xgb.DMatrix(X_test)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    # Train the model! We pass in a max of 1,600 rounds (with early stopping after 70)
    # and the custom metric (maximize=True tells xgb that higher metric is better)
    mdl = xgb.train(params, d_train, 2000, watchlist, early_stopping_rounds=100, feval=gini_xgb, maximize=True, verbose_eval=100)
    
    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    

[Fold 1/5]
[0]	train-error:0.131574	valid-error:0.13671	train-gini:0.33388	valid-gini:0.197909
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-error:0.131574	valid-error:0.13671	train-gini:0.464115	valid-gini:0.272774
Stopping. Best iteration:
[94]	train-error:0.131574	valid-error:0.13671	train-gini:0.460637	valid-gini:0.275081

[Fold 1/5 Prediciton:]
[Fold 2/5]
[0]	train-error:0.131574	valid-error:0.13671	train-gini:0.33388	valid-gini:0.197909
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-error:0.131574	valid-error:0.13671	train-gini:0.464115	valid-gini:0.272774
Stopping. Best iteration:
[94]	train-error:0.131574	valid-error:0.13671	train-gini:0.460637	valid-gini:0.275081

[Fold 2/5 Prediciton:]
[Fold 3/5]
[0]	train-error:0.131574	valid-error:0.13671	train-gini:0.333

In [14]:
p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)

In [15]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [16]:
len(p_test)

15021

In [17]:
sub['target'] = p_test

In [18]:
subd = sub.to_csv('1.csv', index= False)